In [22]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soniakashyap/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
145,**WARNING: POSSIBLE SPOILER**<br /><br />If yo...,negative
806,"At the time, ""My Left Foot"" was the little mov...",positive
911,"Seeing this movie, as I just did for the first...",negative
289,Writer/Director Peter Greenaway cements his ti...,negative
370,"Men, do I love police movies filled with actio...",negative


In [24]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [26]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/soniakashyap/nltk_data...


True

In [27]:
df = normalize_text(df)
df.head()

,review,sentiment
145,warning possible spoiler br br get extremely u...,negative
806,time left foot little movie could hugely popul...,positive
911,seeing movie first time turner classic list da...,negative
289,writer director peter greenaway cement title h...,negative
370,men love police movie filled action shooting c...,negative


In [28]:
df['sentiment'].value_counts()

sentiment
negative    264
positive    236
Name: count, dtype: int64

In [29]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [30]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
145,warning possible spoiler br br get extremely u...,0
806,time left foot little movie could hugely popul...,1
911,seeing movie first time turner classic list da...,0
289,writer director peter greenaway cement title h...,0
370,men love police movie filled action shooting c...,0


In [31]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [32]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [37]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/soniakashyap001/MLOpsSentimentAnalysis.mlflow')
dagshub.init(repo_owner='soniakashyap001', repo_name='MLOpsSentimentAnalysis', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "soniakashyap001/MLOpsSentimentAnalysis"

2025-07-22 20:04:32,623 - INFO - Initialized MLflow to track repo "soniakashyap001/MLOpsSentimentAnalysis"


Repository soniakashyap001/MLOpsSentimentAnalysis initialized!

2025-07-22 20:04:32,624 - INFO - Repository soniakashyap001/MLOpsSentimentAnalysis initialized!


<Experiment: artifact_location='mlflow-artifacts:/c788d371b70f42b1b85995e31734597c', creation_time=1753239209061, experiment_id='0', last_update_time=1753239209061, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [38]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-07-22 20:04:32,993 - INFO - Starting MLflow run...
2025-07-22 20:04:33,170 - INFO - Logging preprocessing parameters...
2025-07-22 20:04:33,783 - INFO - Initializing Logistic Regression model...
2025-07-22 20:04:33,784 - INFO - Fitting the model...
2025-07-22 20:04:33,801 - INFO - Model training complete.
2025-07-22 20:04:33,802 - INFO - Logging model parameters...
2025-07-22 20:04:33,987 - INFO - Making predictions...
2025-07-22 20:04:33,988 - INFO - Calculating evaluation metrics...
2025-07-22 20:04:33,997 - INFO - Logging evaluation metrics...
2025-07-22 20:04:34,646 - INFO - Saving and logging the model...
/Users/soniakashyap/Documents/MLOpsSentimentAnalysis/atlas/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setup